In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import hvplot.xarray 
import panel as pn
from thredds_crawler.crawl import Crawl
import datetime as dt
from scipy.interpolate import interp1d,interp2d
from bokeh.models import  FixedTicker

In [ ]:
hv.extension('bokeh')
pn.extension()

In [ ]:
def prep_mod(url):
    #opens model file from url as dataset
    mod = xr.open_dataset(url)
    
    #calculate height axis for the model, works for IFS
    mod['height'] = mod.zg - mod.zghalf[:, -1]
    
    #select relevant variable and height axis
    mod_ds = mod[['height','ta']]
    
    #flip the height axis and convert to dataframe
    mod_ds = mod_ds.reindex(level=list(reversed(mod_ds.level)))
    tot_df = mod_ds.to_dataframe()
    
    #resample with 6 hour frequency, to match sond frequency 
    #and remove first point in resulting dataframe
    #as it is at point 00:00:00, and the first model point when resampled should occur 
    #at 01:00:00
    df_res = tot_df.swaplevel().unstack().resample('6H').asfreq()
    df_res = df_res[1:]
    
    return df_res
    
def interpolate_mod(df_res,height):
    #create a dummy dataframe with the correct dimensions to hold the results
    time = df_res.index.values
    dummy = pd.DataFrame('NaN',height,time)

    #for each timestep - read for each height axis, interpolate to the new axis
    #then add to the dummy dataframe 
    for i in df_res.index.values:
        temp = df_res.loc[i].unstack().T
        height_model = temp['height']
        temperature = temp['ta']

        interpolated = interp1d(height_model,temperature,fill_value="extrapolate")(height)
        dummy[i] = interpolated
    
    
    return dummy

In [ ]:
def calc_err(mod,obs,hours):
    #get model start and end times
    st = mod.columns.values[0]
    et = mod.columns.values[-1]
    
    #select relevant slice of observational data and reformat 
    #to match model format
    time_obs = obs.sel(time=slice(st, et)).to_dataframe().unstack()
    time_obs.columns = time_obs.columns.droplevel()
    time_obs = time_obs.T

    #calculate error by combining the model and observation dataframes
    #using a set function
    comb_func = lambda x,y: x-y
    err = mod.combine(time_obs,comb_func)
    
    #convert to common time axis based on how many hours into the 3 
    #day forcast the point is
    err.columns = hours
    
    return err


def calc_median(stacked_errors,hours):
    medians = []
    #calculate median for each timestep 
    for hour in hours:
        hour = hour -1
        values_per_time = stacked_errors[:,hour,:]

        medians.append(np.nanmedian(values_per_time,axis=0))
    
    #create new 2D dataset with index height and time, containing
    #the median error 
    plottable= np.stack(medians).T
    
    return plottable


In [ ]:
#create list of urls
catalog = "https://thredds.met.no/thredds/catalog/alertness/YOPP_supersite/ifs-ecmwf/sodankyla/catalog.html"
c = Crawl(catalog, select=['.*20180[23]..00\.nc'])
urls_mod = []
for d in c.datasets:
    for s in d.services:
        if s.get("service").lower() == "opendap":
            urls_mod.append(s.get("url")+"?zg,zghalf,ta,time,level")

urls_mod.sort()

#open observation dataset
obs = xr.open_dataset("sodankyla_sondes_20180201-20180331.nc")['ta']

#set parameters and create list for results
errors = [] 
height = obs['height'].values
hours = np.arange(1,13)


#calculate error for all model urls in list
for url in urls_mod:
    df_res = prep_mod(url)
    dummy = interpolate_mod(df_res,height)
    err = calc_err(dummy,obs,hours)
    errors.append(err.T)
    

#stack the calculated errors along new model number axis
#creating 3D datset indexed by time, height and forcast number
stacked_errors = np.stack(errors)

plottable = calc_median(stacked_errors,hours)




In [ ]:
def plot_err(plottable):
    plt.figure(figsize=(10,5))
    
    #set values for time axis
    time = np.arange(0,100,6)[1:13]
    #create data array with median data as well as height and forcast_lenght
    ds = xr.DataArray(plottable,dims=['height','forcast_length'],coords=dict(height = height,forcast_length=time
    ))
    
    #plot the data
    plot = plt.contourf(ds.forcast_length,ds.height,ds.values)
    
    #set plot labels, colors etc
    degree_sign = u"\N{DEGREE SIGN}"
    plt.colorbar(plot,label=degree_sign+"C")
    
    top = cm.get_cmap('Blues', 127)
    bottom = cm.get_cmap('Reds', 127)

    newcolors = np.vstack((top(np.linspace(0, 1, 127)),
                       bottom(np.linspace(0, 1, 127))))
    
    
    plot.set_cmap(cmap = ListedColormap(newcolors, name='OrangeBlue'))
    plt.xticks()
    plt.xlabel("Forcast length (h)")
    plt.ylabel("Altitude (m)")
    plt.title("Median forcast error of the temperature\nSite: Sodankyla, Model: ECMWF - IFS")
    plt.show()
    

In [ ]:
plot_err(plottable)